In [ ]:
#!pip install keras-unet

In [ ]:
import os
import cv2
import time
import shutil
import random
import inspect
#import imageio as im
import numpy as np
import mahotas as mh
from PIL import Image
from tabulate import tabulate
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
#from skimage import measure, filters

In [ ]:
# original directory path and working directory path for images
data = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/raw_files"
working_dir = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir"
######################################################################################################
experiment = "test3" 
######################################################################################################
real = f"{working_dir}/{experiment}/real_images"
#########################################################################################################

directories = {
    "normalized": f"{working_dir}/{experiment}/normalized_images",
    "cropped": f"{working_dir}/{experiment}/cropped_images",
    "npy": f"{working_dir}/{experiment}/pre_processing/npy",
    "originals": f"{working_dir}/{experiment}/pre_processing/originals",
    "masks": f"{working_dir}/{experiment}/pre_processing/masks",
    "test": f"{working_dir}/{experiment}/pre_processing/test",
    "s": f"{working_dir}/{experiment}/S",
    "crop_original": f"{working_dir}/{experiment}/pre_processing/crop_originals",
    "crop_masks": f"{working_dir}/{experiment}/pre_processing/crop_masks",
    "op": f"{working_dir}/{experiment}/post_processing/original_prediction",
    "pp": f"{working_dir}/{experiment}/post_processing/processed_prediction"
    
}

In [ ]:
npy = f"{working_dir}/{experiment}/pre_processing/npy1"
machine = f"{working_dir}/{experiment}/pre_processing/machine"

In [ ]:
import keras
from keras_unet.models import custom_unet

In [ ]:
imagedata = np.load(os.path.join(npy, 'original.npz'))
labeldata = np.load(os.path.join(npy, 'mask.npz'))

In [ ]:
images_file = imagedata['data']
images_name = imagedata['names']

In [ ]:
len (images_name)


In [ ]:
labels_file = labeldata['data']
labels_name = labeldata['names']


#labels_file = os.path.join(npy, 'mask.npy')
#test_files = os.path.join(npy, 'test.npy')

In [ ]:
images = images_file
labels = labels_file
#test = np.load(test_file)

In [ ]:
images.shape

In [ ]:
labels.shape

In [ ]:
 # full dataset does not have the last channel
images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)
#test = test.reshape(test.shape[0],test.shape[1],test.shape[2],1)

In [ ]:
images.shape

In [ ]:
labels.shape

In [ ]:
p = np.random.permutation(len(images))

In [ ]:
images = images[p]
labels = labels[p]
#test = test[p]

In [ ]:
labels = labels.astype(np.float64)

In [ ]:
images = images.astype(np.float64)
for i in range(images.shape[0]):
    images[i] = (images[i] - images[i].min()) / (images[i].max() - images[i].min()) # normalize individually


In [ ]:
# # Calculate the indices for the splits
# num_images = len(images)
# num_labels = len(labels)
# train_x = int(num_images * 0.6)
# train_y = int(num_labels * 0.6)
# val_x = int(num_images * 0.85)
# val_y = int(num_labels * 0.85)

In [ ]:
# X_train = images[:train_x]
# y_train = labels[:train_y]
# X_val = images[train_x:val_x]
# y_val = labels[train_y:val_y]
# X_test = images[val_x:]
# y_test = labels[val_y:]

#X_test = test[80:]
#y_test = test[80:]

In [ ]:
X_train = images[0:1200]
y_train = labels[0:1200]
X_val = images[1201:1700]
y_val = labels[1201:1700]
X_test = images[0:1812]
y_test = labels[0:1812]

In [ ]:
y_train.shape

In [ ]:
def randomimage(abc):
    jk = 1187
    binary_mask = y_train[jk]#.squeeze()
    # Plot the binary mask
    plt.imshow(binary_mask, cmap='gray')
    plt.show()
    labeled, number = mh.label(binary_mask)
    sizes = mh.labeled.labeled_size(labeled)
    print(sizes)
    a =sizes.max()
    print('size-max',a)
    b =sizes.min()
    print('size-min',b)
    c = binary_mask.shape
    print('shape',c)

In [ ]:
randomimage(897)

In [ ]:
###################   LOADING MODEL   ######################

In [ ]:
opt = keras.optimizer_v1.Adam(lr=0.01)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from keras.metrics import MeanIoU

n_classes = 2 # number of classes in your segmentation masks
IoU = MeanIoU(num_classes=n_classes)

model = Sequential()
# Encoder
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(512, 512,1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

# Decoder
model.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), activation='relu', padding='same'))
model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same'))
model.add(Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[IoU])

In [ ]:
########### IMPORTS #############

In [ ]:
from keras import regularizers
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras_unet.utils import plot_segm_history, plot_imgs
from skimage import measure

import skimage.filters as filters
import skimage.measure as measure
from sklearn.metrics import classification_report
import keras.optimizers
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

In [ ]:
########################   FUNCTIONS ######################

In [ ]:
def labels(binary_mask):
    # Calculate number_1
    labeled, num = mh.label(binary_mask)
    print("Number_1:", num)

    # Calculate region sizes
    region_sizes = measure.regionprops(labeled, intensity_image=binary_mask)
    sizes = [region.area for region in region_sizes]

    # Find min_region_size
    sizes.sort()
    large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] < numbi), len(sizes))
    #large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] > 100), len(sizes))
    min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
    print("Min_region_size:", min_region_size)
    #print ('sizes:',sizes)
    a = len(sizes)
    print ('length of sizes =',a)

    # Calculate large_regions
    num_large_regions = 0
    for region in region_sizes:
        if region.area > min_region_size:
            num_large_regions += 1
    print("Large_regions:", num_large_regions)

    # Calculate threshold
    thresh = filters.threshold_otsu(binary_mask)
    #print(labels_name[jk])
    print("Threshold:", thresh)

In [ ]:
def pros(binary_mask):
    import matplotlib.pyplot as plt
    from io import StringIO
    import sys
    # ...

    fig, axs = plt.subplots(1, 5, figsize=(15, 5))

    # Operation 1
    labeled, number = mh.label(binary_mask)
    sizes = mh.labeled.labeled_size(labeled)
    numbi = int(np.average(sizes))
    large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] < numbi), len(sizes))
    min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
    too_small = np.where(sizes < min_region_size)
    labeled_only_big = mh.labeled.remove_regions(labeled, too_small)
    axs[0].imshow(labeled)
    #axs[0].imshow(labeled_only_big)
    axs[0].set_title('Operation 1')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(labeled_only_big)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[0].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[0].transAxes)

    # Operation 2
    binary_mask = labeled_only_big.copy()
    binary_mask[binary_mask > 0] = 1

    axs[1].imshow(binary_mask)
    axs[1].set_title('Operation 2')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[1].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[1].transAxes)

    # Operation 3
    binary_mask_closed = mh.gaussian_filter(binary_mask, sigma=5)

    axs[2].imshow(binary_mask_closed)
    axs[2].set_title('Operation 3')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask_closed)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[2].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[2].transAxes)

    # Operation 4
    binary_mask= binary_mask.reshape(binary_mask.shape[0],binary_mask.shape[1])
    binary_mask_open = mh.morph.close_holes(binary_mask)

    #axs[3].imshow(binary_mask_open)
    axs[3].set_title('Operation 4')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask_open)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[3].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[3].transAxes)

    # Operation 5
    for i in range(3):
        binary_mask_dialate = mh.morph.dilate(binary_mask_open)

    axs[4].imshow(binary_mask_dialate)
    axs[4].set_title('Operation 5')

    # Capture the output of the labels function
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()

    labels(binary_mask_dialate)

    # Restore stdout
    sys.stdout = old_stdout

    # Display the captured output as text below the subplot
    axs[4].text(0.5, -0.1, mystdout.getvalue(), ha='center', va='top', transform=axs[4].transAxes)

    plt.show()

In [ ]:
def readpaths(directories):
    for key, path in directories.items():
        globals()[key] = path

In [ ]:
readpaths(directories)

In [ ]:
len(y_train)

In [ ]:
jk = 1187

In [ ]:
def randomimage(abc):
    jk = 1187
    #binary_mask = y_train[jk].squeeze()
    # Plot the binary mask
    plt.imshow(binary_mask, cmap='gray')
    plt.show()
    labeled, number = mh.label(binary_mask)
    sizes = mh.labeled.labeled_size(labeled)
    print(sizes)
    a =sizes.max()
    print('size-max',a)
    b =sizes.min()
    print('size-min',b)
    c = binary_mask.shape
    print('shape',c)

In [ ]:
jk = 1087
#binary_mask = y_train[jk].squeeze()
# Plot the binary mask
plt.imshow(binary_mask, cmap='gray')
plt.show()
labeled, number = mh.label(binary_mask)
sizes = mh.labeled.labeled_size(labeled)
print(sizes)
a =sizes.max()
print('size-max',a)
b =sizes.min()
print('size-min',b)
c = binary_mask.shape
print('shape',c)

In [ ]:
labeled, number = mh.label(binary_mask)
sizes = mh.labeled.labeled_size(labeled)
# large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1]), len(sizes))
# min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
# too_small = np.where(sizes < min_region_size)
# labeled_only_big = mh.labeled.remove_regions(labeled, too_small)
# axs[0].imshow(labeled)

In [ ]:
sizes.max()

In [ ]:
sizes.min()

In [ ]:
binary_mask.shape

In [ ]:
np.average(sizes)

In [ ]:
print(sizes)

In [ ]:
max_index = np.argmax(sizes)

In [ ]:
# Remove maximum value
sizes = np.delete(sizes, max_index)

In [ ]:
sizes.max()

In [ ]:
sizes.min()

In [ ]:
a = int(np.average(sizes))
print (a)

In [ ]:
print(sizes)

In [ ]:
numbi = int(np.average(sizes))
large_region_start_index = next((i for i in range(1, len(sizes)) if (sizes[i] - sizes[i-1]) < numbi), len(sizes))
min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
print (min_region_size)

In [ ]:
plt.imshow(binary_mask)

In [ ]:
labeled, number = mh.label(binary_mask)

plt.imshow(labeled)

In [ ]:
binary_mask = labeled.copy()
binary_mask[binary_mask > 0] = 1
   
plt.imshow(binary_mask)

In [ ]:
region_sizes = measure.regionprops(binary_mask, intensity_image=binary_mask)
print(region_sizes)

In [ ]:

labeled, number = mh.label(binary_mask)
sizes = mh.labeled.labeled_size(labeled)
#sizes = np.array([region.area for region in region_sizes], dtype=np.int64)
numbi = int(np.average(sizes))
#print(sizes)

# large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] < numbi), len(sizes))
# print(large_region_start_index)
# min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
# print(min_region_size)


In [ ]:
labeled.shape

In [ ]:
sizes

In [ ]:
import mahotas as mh
from skimage import measure, filters
import matplotlib.pyplot as plt
jk = 87
# Select a single binary mask from y_train
binary_mask = y_train[jk].squeeze()

# Plot the binary mask
plt.imshow(binary_mask, cmap='gray')
plt.show()

# Calculate number_1
labeled, num = mh.label(binary_mask)
print("Number_1:", num)

# Calculate region sizes
region_sizes = measure.regionprops(labeled, intensity_image=binary_mask)
sizes = [region.area for region in region_sizes]

# Find min_region_size
sizes.sort()
large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] > 100), len(sizes))
min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
print("Min_region_size:", min_region_size)

# Calculate large_regions
num_large_regions = 0
for region in region_sizes:
    if region.area > min_region_size:
        num_large_regions += 1
print("Large_regions:", num_large_regions)

# Calculate threshold
thresh = filters.threshold_otsu(binary_mask)
print(labels_name[jk])
print("Threshold:", thresh)
print()
print(labels_name[jk])

In [ ]:
print(sizes)
len(sizes)

In [ ]:
labeled, number = mh.label(binary_mask)
sizes = mh.labeled.labeled_size(labeled)
print (sizes)
len(sizes)
too_small = np.where(sizes < min_region_size)
labeled_only_big = mh.labeled.remove_regions(labeled, too_small)

In [ ]:
binary_mask_closed = mh.gaussian_filter(binary_mask, sigma=5)

plt.imshow(binary_mask_closed)

In [ ]:
binary_mask_open=mh.morph.close_holes(binary_mask)
plt.imshow(binary_mask_open)

In [ ]:
for i in range(3):
    binary_mask_open = mh.morph.dilate(binary_mask_open)
    plt.imshow(binary_mask_open)

In [ ]:
experiment = "test3" 
working_dir = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir"
pp =f"{working_dir}/{experiment}/post_processing/processed_prediction"
op = f"{working_dir}/{experiment}/post_processing/original_prediction"
crop_original =  f"{working_dir}/{experiment}/pre_processing/crop_originals"
crop_masks = f"{working_dir}/{experiment}/pre_processing/crop_masks"

In [ ]:
from keras import regularizers
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras_unet.utils import plot_segm_history, plot_imgs
from skimage import measure

import skimage.filters as filters
import skimage.measure as measure
from sklearn.metrics import classification_report
import keras.optimizers
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

In [ ]:
import mahotas as mh

In [ ]:
import numpy as np

In [ ]:
import cv2
from PIL import Image
from skimage.filters import threshold_otsu

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_train,
                    y_train,
                    batch_size=60,
                    epochs=150,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping])
# Make predictions and apply thresholding
y_pred = model.predict(X_test)

# Apply contrast stretching to the predicted images
new_min_value = 0
new_max_value = 255

for i in range(y_pred.shape[0]): # for i in range(y_pred.shape[0])
    z = images_name[i]
    binary_mask = y_pred[i]
    binary_mask.shape    
    print(binary_mask.shape)
    
    # Apply Otsu's thresholding to binary mask
    thresh = 0.25
    #thresh = threshold_otsu(binary_mask)
    binary_mask_thresh = binary_mask > thresh
    
    sizes_b = mh.labeled.labeled_size(binary_mask_thresh)
    
    
    print(z)
    plt.imshow(binary_mask_thresh)
    plt.show()
    print(binary_mask_thresh.shape)
    print("Sizes for image a", sizes_b)
    
    # Convert binary mask to uint8 format
    binary_mask_uint8 = (binary_mask * 255).astype('uint8')
    
#     labeled12, number = mh.label(binary_mask_uint8)
#     sizes_l12 = mh.labeled.labeled_size(labeled12)
#     print("Sizes for image", sizes_l12)
    
    # Remove any singleto0n dimensions
    binary_mask_uint8 = np.squeeze(binary_mask_uint8)
#     labeled12, number = mh.label(binary_mask_uint82)
#     sizes_l13 = mh.labeled.labeled_size(labeled12)
#     print("Sizes for image binary_mask_uint82", sizes_l13)

    # Select only the first channel if necessary
    if binary_mask_uint8.ndim == 3 and binary_mask_uint8.shape[2] not in [1, 3, 4]:
        binary_mask_uint8 = binary_mask_uint8[..., 0]
        
#     labeled1, number = mh.label(binary_mask_uint8)
#     sizes_l1 = mh.labeled.labeled_size(labeled1)
#     print("Sizes for image binary_mask_uint8", sizes_l1)
    
    
    plt.imshow(binary_mask_uint8)
    plt.show()
    print ('###############')
    # Create Image object from binary mask
    binary_mask_imag = Image.fromarray(binary_mask_uint8)
    # Save binary mask image
    binary_mask_imag.save(f'{op}/{z}')
    print(f'saved:{op}/{z}')
#     save_path = os.path.join(op, z)
#     print("Saving image to:", save_path)    
#     mh.imsave(save_path, binary_mask)

In [ ]:
import cv2
from PIL import Image
from skimage.filters import threshold_otsu
from keras.preprocessing.image import ImageDataGenerator

# # Create an instance of the ImageDataGenerator class
# datagen = ImageDataGenerator(
#     rotation_range=10, # randomly rotate images up to 10 degrees
#     width_shift_range=0.1, # randomly shift images horizontally by up to 10%
#     height_shift_range=0.1, # randomly shift images vertically by up to 10%
#     horizontal_flip=True, # randomly flip images horizontally
#     zoom_range=0.1 # randomly zoom in/out on images by up to 10%
# )

# # Fit the data generator on your training data
# datagen.fit(X_train)

# # Early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# history = model.fit(
#                     datagen.flow(X_train, y_train, batch_size=60),                    
#                     epochs=700,
#                     validation_data=(X_val, y_val),
#                     callbacks=[early_stopping])


# # Save the trained weights to a file
# model.save_weights('new_model1.h5')

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_train,
                    y_train,
                    batch_size=60,
                    epochs=700,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping])
# # Save the trained weights to a file
model.save_weights('new_model1.h5')
# Make predictions and apply thresholding
y_pred = model.predict(X_test)
for i in range(5):
    z = images_name[i]
    binary_mask = y_pred[i]
    plt.imshow(binary_mask)
    plt.show()
    
history.history.keys()
metrics = list(history.history.keys())
plot_segm_history(history, metrics=metrics)   
plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=5)
#     #thresh = threshold_otsu(binary_mask)
#     binary_mask_thresh = binary_mask > thresh    
#     sizes_b = mh.labeled.labeled_size(binary_mask_thresh)
#     binary_mask_uint8 = (binary_mask * 255).astype('uint8')
#     binary_mask_uint8 = np.squeeze(binary_mask_uint8)
#     # Select only the first channel if necessary
#     if binary_mask_uint8.ndim == 3 and binary_mask_uint8.shape[2] not in [1, 3, 4]:
#         binary_mask_uint8 = binary_mask_uint8[..., 0]
#     # Create Image object from binary mask
#     binary_mask_imag = Image.fromarray(binary_mask_uint8)
#     # Save binary mask image
# #     binary_mask_imag.save(f'{op}/{z}')
# #     print(f'saved:{op}/{z}')

In [ ]:
def create_model():
    n_classes = 2 # number of classes in your segmentation masks
    IoU = MeanIoU(num_classes=n_classes)
    model = Sequential()
    # Encoder
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(512, 512,1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    # Decoder
    model.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), activation='relu', padding='same'))
    model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same'))
    model.add(Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same'))
#         model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[IoU])
    return model

In [ ]:
def wtrain(number):
    import cv2
    from PIL import Image
    from skimage.filters import threshold_otsu
    from keras.preprocessing.image import ImageDataGenerator
    al = 1
    for k in range(number):
        # create a new model with the same architecture as the original model
        new_model = create_model()
        path = '/raid/mpsych/RISTERLAB/kiran/flyem/K_Experiments/models'
        # load the previously saved weights into the new model
        new_model.load_weights(f'{path}/new_model{al}.h5')
        print (f'loaded model : new_model{al}.h5')
        # compile the new model
        IoU = MeanIoU(num_classes=2)
        
        # compile the new model
        new_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[IoU])
        # Early stopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        history = new_model.fit(X_train,
                            y_train,
                            batch_size=60,
                            epochs=700,
                            validation_data=(X_val, y_val),
                            callbacks=[early_stopping])

        bl = (al+1)
        # save the updated weights of the new model
        new_model.save_weights(f'{path}/new_model{bl}.h5')
        
        print (f'saved model : new_model{bl}.h5')
        # Make predictions and apply thresholding
        y_pred = new_model.predict(X_test)



        for i in range(5):
            z = images_name[i]
            binary_mask = y_pred[i]
            plt.imshow(binary_mask)
            plt.show()

        history.history.keys()
        metrics = list(history.history.keys())
        plot_segm_history(history, metrics=metrics)   
        plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=5)
        al += 1
        print ('completed iteration number',k,'The value of al is ',al)
        print ('#############################')
        print()
        print()
        print()

In [ ]:
!pwd

In [ ]:
wtrain(100)

In [ ]:
al =99
ran=15
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt


# create a new model with the same architecture as the original model
new_model = create_model()

path = '/raid/mpsych/RISTERLAB/kiran/flyem/K_Experiments/models'
# load the previously saved weights into the new model
new_model.load_weights(f'{path}/new_model{al}.h5')
# load the saved weights into the new model
p = np.random.permutation(len(images))
images = images[p]
L_test = images[:ran]    
X_new_subset = L_test
# make predictions on new data
y_pred = new_model.predict(X_new_subset)
# plot the predictions for the first 10 images
for i in range(ran):    
    binary_mask = y_pred[i]
    plt.imshow(binary_mask)
    plt.show()
    
history.history.keys()
metrics = list(history.history.keys())
plot_segm_history(history, metrics=metrics)   
plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=5)

In [ ]:
import matplotlib.pyplot as plt

# select the first 10 images from X_new
X_new_subset = X_new[:10]

# make predictions on the subset of data
y_pred_subset = new_model.predict(X_new_subset)

# plot the predictions for the first 10 images
plt.plot(y_pred_subset)
plt.show()

In [ ]:
import cv2
from PIL import Image
from skimage.filters import threshold_otsu
from keras.preprocessing.image import ImageDataGenerator

# create a new model with the same architecture as the original model
new_model = create_model()

# load the previously saved weights into the new model
new_model.load_weights('new_model1.h5')

# compile the new model
new_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[IoU])

# Create an instance of the ImageDataGenerator class
datagen = ImageDataGenerator(
    rotation_range=10, # randomly rotate images up to 10 degrees
    width_shift_range=0.1, # randomly shift images horizontally by up to 10%
    height_shift_range=0.1, # randomly shift images vertically by up to 10%
    horizontal_flip=True, # randomly flip images horizontally
    zoom_range=0.1 # randomly zoom in/out on images by up to 10%
)
# Fit the data generator on your training data
datagen.fit(X_train)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(
                    datagen.flow(X_train, y_train, batch_size=60),                    
                    epochs=350,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping])
# save the updated weights of the new model
new_model.save_weights('new_model1.h5')

In [ ]:
##################################################################################################

In [ ]:
def processing(binary_mask):
    labeled, number = mh.label(binary_mask)
    sizes = mh.labeled.labeled_size(labeled)
    print ('sizes:',sizes)
    len(sizes)
    too_small = np.where(sizes < min_region_size)
    labeled_only_big = mh.labeled.remove_regions(labeled, too_small)

    ##############

    binary_mask = labeled_only_big.copy()
    binary_mask[binary_mask > 0] = 1

    print('labelled only big')
    plt.imshow(binary_mask)
    labels(binary_mask)
    #######

    binary_mask_closed = mh.gaussian_filter(binary_mask, sigma=5)
    print('gassuian filter sigma =5')
    plt.imshow(binary_mask_closed)
    labels(binary_mask_closed)

    #######
    binary_mask_open=mh.morph.close_holes(binary_mask)
    plt.imshow(binary_mask_open)
    labels(binary_mask_open)


    ##########
    for i in range(3):
        binary_mask_open = mh.morph.dilate(binary_mask_open)
        print('dilate range of 3')
        plt.imshow(binary_mask_open)
        labels(binary_mask_open)

In [ ]:
def nt_early_pp(batch,epochs,plot,thres):
    import cv2
    from PIL import Image

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    history = model.fit(X_train,
                        y_train,
                        batch_size=batch,
                        epochs=epochs,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Make predictions and apply thresholding
    y_pred = model.predict(X_test)
    y_pred = (y_pred > thres).astype(np.uint8)

    # Calculate precision, recall, and F1 score
    #report = classification_report(y_test.flatten(), y_pred.flatten(), target_names=['background', 'circle'], output_dict=True)
    #print('Classification report:\n', report)

    # Apply contrast stretching to the predicted images
    new_min_value = 0
    new_max_value = 255

    for i in range(y_pred.shape[0]):
        z = images_name[i]
        image = Image.fromarray((y_pred[i].squeeze() * 255).astype(np.uint8))
        min_value, max_value = image.getextrema()
        scale = (new_max_value - new_min_value) / (max_value - min_value)
        image = image.point(lambda i: (i - min_value) * scale + new_min_value)
        y_pred[i] = np.expand_dims(np.array(image), axis=-1) / 255
        print(z)
        binary_mask = y_pred[i]
        plt.imshow(binary_mask, cmap='gray')
        plt.show()
        #processing(binary_mask)
        
        #
        
        
        
    # Plotting results
    from keras_unet.utils import plot_segm_history
    # shutil.copy(os.path.join(path1, z), os.path.join(path2, z))
    # save_path = os.path.join(save_dir, image_name[i])
    # print("Saving image to:", save_path)
    # cv2.imwrite(save_path, y_pred)

#     history.history.keys()
#     metrics = list(history.history.keys())
#     plot_segm_history(history, metrics=metrics)   
#     plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=plot)



In [ ]:
labeled, number = mh.label(binary_mask)
    sizes = mh.labeled.labeled_size(labeled)
    #print ('sizes:',sizes)
    a = len(sizes)
    print ('length of sizes =',a)
    too_small = np.where(sizes < min_region_size)
    labeled_only_big = mh.labeled.remove_regions(labeled, too_small)

    ##############

    binary_mask = labeled_only_big.copy()
    binary_mask[binary_mask > 0] = 1

    print('labelled only big')
    plt.imshow(binary_mask)
    labels(binary_mask)
    #######

    binary_mask_closed = mh.gaussian_filter(binary_mask, sigma=5)
    print('gassuian filter sigma =5')
    plt.imshow(binary_mask_closed)
    labels(binary_mask_closed)

    #######
    binary_mask_open=mh.morph.close_holes(binary_mask)
    plt.imshow(binary_mask_open)
    labels(binary_mask_open)


    ##########
    for i in range(3):
        binary_mask_open = mh.morph.dilate(binary_mask_open)
        print('dilate range of 3')
        plt.imshow(binary_mask_open)
        labels(binary_mask_open)

In [ ]:

binary_mask = labeled_only_big.copy()
binary_mask[binary_mask > 0] = 1

#binary_mask = mh.morph.close(binary_mask)
# Plot the binary mask
plt.imshow(binary_mask, cmap='gray')
plt.show()

# Calculate number_1
labeled, num = mh.label(binary_mask)
print("Number_1:", num)

# Calculate region sizes
region_sizes = measure.regionprops(labeled, intensity_image=binary_mask)
sizes = [region.area for region in region_sizes]

# Find min_region_size
sizes.sort()
large_region_start_index = next((i for i in range(1,len(sizes)) if sizes[i] - sizes[i-1] > 100), len(sizes))
min_region_size = (sizes[large_region_start_index-1] + sizes[large_region_start_index]) // 2
print("Min_region_size:", min_region_size)

# Calculate large_regions
num_large_regions = 0
for region in region_sizes:
    if region.area > min_region_size:
        num_large_regions += 1
print("Large_regions:", num_large_regions)

# Calculate threshold
thresh = filters.threshold_otsu(binary_mask)
print("Threshold:", thresh)

In [ ]:
######################### POST PROCESSING###############

In [ ]:
import cv2
import numpy as np

# Load the binary image
image = cv2.imread('image.png', 0)

# Find the contours in the image
contours, _ = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Calculate the sizes and thicknesses of the contours
sizes = []
thicknesses = []
for contour in contours:
    size = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    thickness = 4 * np.pi * size / (perimeter ** 2)
    sizes.append(size)
    thicknesses.append(thickness)

# Calculate the mean and standard deviation of the sizes and thicknesses
mean_size = np.mean(sizes)
std_size = np.std(sizes)
mean_thickness = np.mean(thicknesses)
std_thickness = np.std(thicknesses)

# Set the minimum size and thickness for the contours
min_size = mean_size - std_size
min_thickness = mean_thickness - std_thickness

# Calculate the distances between the contours
distances = []
for contour1 in contours:
    for contour2 in contours:
        if contour1 is not contour2:
            dist = cv2.pointPolygonTest(contour1, tuple(contour2[0][0]), True)
            distances.append(abs(dist))

# Calculate the median of the distances
median_distance = np.median(distances)

# Set the distance threshold for the pointPolygonTest function
distance_threshold = median_distance

# Create a mask to store the filtered contours
mask = np.zeros_like(image)

# Iterate through the contours
for contour in contours:
    # Calculate the size and thickness of the contour
    size = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    thickness = 4 * np.pi * size / (perimeter ** 2)

    # Check if the contour is close to any larger regions
    close_to_larger_region = False
    for other_contour in contours:
        if cv2.contourArea(other_contour) > size:
            dist = cv2.pointPolygonTest(other_contour, tuple(contour[0][0]), True)
            if abs(dist) < distance_threshold:
                close_to_larger_region = True
                break

    # If the contour is large enough or close to a larger region or thick enough, add it to the mask
    if size > min_size or close_to_larger_region or thickness > min_thickness:
        cv2.drawContours(mask, [contour], -1, 255, -1)

# Apply the mask to the image to remove the small white dots
result = cv2.bitwise_and(image, mask)

# Show the result
cv2.imshow('Result', result)
cv2.waitKey(0)